In [1]:
import sys
sys.path.append("/experiments_fbs/")

In [2]:
import pickle
import pandas as pd
import numpy as np
import openml
import glob
from gama import GamaClassifier

from sklearn.cluster import KMeans

from run_pipeline import *
from automl_openml import AutomlExecutor
from collect_metadata import MetaDataSet
from clustering import MetaDataCluster
from portfolio import PortFolioBuilder

In [3]:
data_directory = "../data/batch_results"
csv_files = glob.glob(f"{data_directory}/*.{'csv'}")
df = pd.concat([pd.read_csv(f, index_col = 0) for f in csv_files])
df.dropna(inplace = True)
test_indexes = df.sample(30, replace = False).index
train_indexes = df.drop(test_indexes, axis = 0).index

In [4]:
df.loc[test_indexes]

,"Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', VarianceThreshold(threshold=0.2)),\n ('0',\n RandomForestClassifier(criterion='entropy',\n max_features=0.6000000000000001,\n min_samples_leaf=11,\n min_samples_split=11))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n DecisionTreeClassifier(criterion='entropy', max_depth=9,\n min_samples_leaf=11,\n min_samples_split=8))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0', KNeighborsClassifier(n_neighbors=43))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n ExtraTreesClassifier(bootstrap=True, max_features=0.45,\n min_samples_leaf=7))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', VarianceThreshold(threshold=0.15000000000000002)),\n ('0', MultinomialNB(alpha=0.01, fit_prior=False))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n RandomForestClassifier(bootstrap=False, criterion='entropy',\n max_features=0.3, min_samples_leaf=11,\n min_samples_split=18))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', PCA(iterated_power=3, svd_solver='randomized')),\n ('0', LogisticRegression(C=0.5))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n RandomForestClassifier(bootstrap=False, criterion='entropy',\n max_features=0.1, min_samples_leaf=2,\n min_samples_split=3))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n GradientBoostingClassifier(max_depth=5,\n max_features=0.15000000000000002,\n min_samples_leaf=12,\n min_samples_split=4,\n subsample=0.7500000000000001))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', FastICA(tol=0.4, whiten='unit-variance')),\n ('0',\n ExtraTreesClassifier(criterion='entropy', max_features=0.4,\n min_samples_leaf=18,\n min_samples_split=9))])",...,"Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0', LogisticRegression(C=0.1))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', SelectPercentile(percentile=80)),\n ('0',\n DecisionTreeClassifier(max_depth=6, min_samples_leaf=3,\n min_samples_split=12))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', PCA(iterated_power=2, svd_solver='randomized')),\n ('0',\n GradientBoostingClassifier(learning_rate=0.001, max_depth=9,\n max_features=0.1,\n min_samples_leaf=3,\n min_samples_split=3,\n subsample=0.4))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', SelectPercentile(percentile=86)),\n ('0',\n ExtraTreesClassifier(criterion='entropy', max_features=0.8,\n min_samples_leaf=12,\n min_samples_split=14))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n ExtraTreesClassifier(bootstrap=True, max_features=0.45,\n min_samples_leaf=11,\n min_samples_split=11))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', FastICA(tol=0.55, whiten='unit-variance')),\n ('0',\n RandomForestClassifier(max_features=0.3, min_samples_leaf=19,\n min_samples_split=9))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', VarianceThreshold(threshold=0.35000000000000003)),\n ('0', GaussianNB())])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('0',\n ExtraTreesClassifier(bootstrap=True, criterion='entropy',\n max_features=0.55, min_samples_leaf=5,\n min_samples_split=10))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', SelectFwe(alpha=0.025)),\n ('0',\n GradientBoostingClassifier(max_depth=4,\n max_features=0.35000000000000003,\n min_samples_split=4,\n subsample=0.9500000000000001))])","Pipeline(steps=[('imputation', SimpleImputer(strategy='median')),\n ('1', PolynomialFeatures(include_bias=False)),\n ('0',\n ExtraTreesClassifier(max_features=0.9000000000000001,\n min_samples_leaf=19,\n min_samples_split=10))])"
0,,,,,

In [5]:
#create meta data set
# mds = MetaDataSet(train_indexes)
# meta_dataset = mds.create_meta_dataset()
# meta_dataframe = pd.DataFrame(index = train_indexes, data = meta_dataset)
meta_dataframe = pd.read_csv("meta_data.csv", index_col = 0)

In [6]:
def indexes_per_cluster(labels):
    vals, inverse, count = np.unique(labels, return_inverse=True,
                                return_counts=True)

    idx_vals_repeated = np.where(count >= 1)[0]
    vals_repeated = vals[idx_vals_repeated]

    rows, cols = np.where(inverse == idx_vals_repeated[:, np.newaxis])
    _, inverse_rows = np.unique(rows, return_index=True)
    cluster_indexes = np.split(cols, inverse_rows[1:])
    return cluster_indexes

In [7]:
#create cluster
n_clusters = 3
clustering_obj = KMeans(n_clusters = n_clusters, init = 'random', n_init = 'auto')
metadata_cluster = MetaDataCluster(meta_dataframe)
cluster = metadata_cluster.compute_clusters(clustering_obj)
cluster_labels = cluster.labels_

In [8]:
cluster_indexes = indexes_per_cluster(cluster_labels)

In [9]:
builder = PortFolioBuilder(size = 15)
portfolio_dict = dict()
for label, c_index in enumerate(cluster_indexes):
    portfolio = builder.build_portfolio(df, train_indexes[c_index])
    portfolio_dict[label] = portfolio.index
    #store portfolio for reference (as dict)

In [10]:
evaluations = []
pipeline_runner = PipelineExecutor()
for test_index in test_indexes:
    dataset = openml.datasets.get_dataset(test_index)
    meta_data = pd.DataFrame.from_dict(orient = "index", data = dataset.qualities).T
    meta_data = meta_data.combine_first(pd.DataFrame(meta_dataframe.mean(axis = 0)).T).values
    X, y, _, _ = dataset.get_data(dataset_format="dataframe", target = dataset.default_target_attribute)
    evaluations_individual = []
    cluster_assignment = cluster.predict(meta_data)
    portfolio = portfolio_dict[cluster_assignment[0]]
    for pipeline in portfolio:
        sklearn_pipeline = eval(pipeline)
        x_enc, _ = pipeline_runner.basic_encoding(X, is_classification=True)
        accuracy = pipeline_runner.evaluate_pipeline(sklearn_pipeline, x_enc, y)
        evaluations_individual.append(accuracy)
    evaluations.append(max(evaluations_individual))

df = pd.DataFrame(index = test_indexes, data = evaluations, columns = portfolio)


Could not download file from http://openml1.win.tue.nl/dataset44389/dataset_44389.pq: Bucket does not exist or is private.


'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'


Could not download file from http://openml1.win.tue.nl/dataset44556/dataset_44556.pq: Bucket does not exist or is private.


'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'


/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  

...

Could not download file from http://openml1.win.tue.nl/dataset44544/dataset_44544.pq: Bucket does not exist or is private.


'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'
'str' object has no attribute 'fit'


Could not download file from http://openml1.win.tue.nl/dataset44774/dataset_44774.pq: Bucket does not exist or is private.
/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/home/wichert/Master DS&AI/master_thesis/fstudy_env/lib/python3.10/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


'str' object has no attribute 'fit'


KeyboardInterrupt: 

In [ ]:
#get best performance from df

In [ ]:
gamaclassifier = GamaClassifier(max_total_time = 150, store = 'nothing')
automl_instance = AutomlExecutor(gamaclassifier)
accuracies_gama = []
for test_index in test_indexes:
    accuracy = automl_instance.run_automl(test_index)
    print(f"accuracy for run on {test_index} is {accuracy}")
    accuracies_gama.append(accuracy)